## Rental hedonics estimation

Sam Maurer, July 2015

In [2]:
%load_ext autoreload
%autoreload 2
if 'sim' not in globals():
    import os; os.chdir('..')
import models
import urbansim.sim.simulation as sim
import pandas as pd
pd.set_option('display.max_columns', 500)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


### 1. Look at the Craigslist data

In [3]:
cl = sim.get_table('craigslist').to_frame()
cl.head(5)

Filling column shape_area with value 604.336158108 (0 values)
Filling column zone_id with value 1178 (0 values)


,neighborhood,price,bedrooms,date,sqft_per_unit,longitude,latitude,price_sqft,node_id,zone_id
0,twin peaks / diamond hts,4500,2,2014-05-11,1200,-122.438300,37.745000,3.750000,300155,800
1,sunnyvale,2650,2,2014-05-11,1040,-122.008131,37.353699,2.548077,143129,960
2,glen park,3100,2,2014-05-11,1000,-122.439743,37.731584,3.100000,125905,1031
3,redwood city,1850,1,2014-05-11,792,-122.234294,37.491715,2.335859,143879,963
4,walnut creek,1325,1,2014-05-11,642,-122.087751,37.923448,2.063863,70923,888


In [4]:
cl.describe()

,price,bedrooms,sqft_per_unit,longitude,latitude,price_sqft,node_id,zone_id
count,73644.000000,73644.000000,73644.00000,73644.000000,73644.000000,73644.000000,73644.000000,73603.000000
mean,3806.642944,1.858305,1196.62111,-122.108903,37.639416,4.697802,156857.443906,890.963031
std,146802.055710,0.932134,31192.63512,1.359750,0.863417,183.768413,100783.186320,82.436500
min,1.000000,1.000000,1.00000,-159.609375,-85.561269,0.000154,62.000000,747.000000
25%,1831.000000,1.000000,720.00000,-122.286807,37.383128,2.055256,80038.000000,831.000000
50%,2330.500000,2.000000,904.00000,-122.050473,37.601374,2.587519,149462.000000,879.000000
75%,2986.000000,2.000000,1160.00000,-121.952270,37.844037,3.135714,190924.000000,956.000000
max,26722744.000000,8.000000,8388607.00000,146.425781,79.858833,31145.389277,354659.000000,1157.000000


### 2. Set up the network vars

In [5]:
sim.run(["neighborhood_vars"])

Running model 'neighborhood_vars'
Computing accessibility variables
Computing sum_residential_units
Filling column building_type_id with value 1 (0 values)
Filling column residential_units with value 0 (0 values)
Filling column year_built with value 1966.0 (0 values)
Filling column non_residential_sqft with value 0 (0 values)
Removed 4 rows because they contain missing values
Computing sum_nonresidential_units
Removed 4 rows because they contain missing values
Computing ave_sqft_per_unit
Removed 2 rows because they contain missing values
Computing ave_lot_size_per_unit
Removed 2 rows because they contain missing values
Computing population
Removed 79699 rows because they contain missing values
Computing poor
Removed 53966 rows because they contain missing values
Computing renters
Removed 4919 rows because they contain missing values
Computing sfdu
Removed 1 rows because they contain missing values
Computing ave_hhsize
Removed 79699 rows because they contain missing values
Computing job

### 3. Estimate a rental listings hedonic

In [ ]:
# The model expression is in rrh.yaml; price_per_sqft is the asking monthly rent per square 
# foot from the Craigslist listings. Price, sqft, and bedrooms are specific to the unit, 
# while all the other variables are aggregations at the node or zone level. Note that we 
# can't use bedrooms in the simulation stage because it's not in the unit data.

In [6]:
sim.run(["rrh_estimate"])

Running model 'rrh_estimate'
                            OLS Regression Results                            
Dep. Variable:     np.log(price_sqft)   R-squared:                       0.497
Model:                            OLS   Adj. R-squared:                  0.497
Method:                 Least Squares   F-statistic:                     6566.
Date:                Thu, 06 Aug 2015   Prob (F-statistic):               0.00
Time:                        22:46:35   Log-Likelihood:                -5272.0
No. Observations:               73168   AIC:                         1.057e+04
Df Residuals:                   73156   BIC:                         1.068e+04
Df Model:                          11                                         
Covariance Type:            nonrobust                                         
                               coef    std err          t      P>|t|      [95.0% Conf. Int.]
----------------------------------------------------------------------------------------

In [ ]:
# to try variations, create a new yaml file and run this to register it

@sim.model()
def rh_cl_estimate_NEW(craigslist, aggregations):
    return utils.hedonic_estimate("rrh_NEW.yaml", craigslist, aggregations)

sim.run(["rrh_estimate_NEW"])

### 4. Compare to current sales hedonic

In [7]:
sim.run(["rsh_estimate"])

Running model 'rsh_estimate'
                              OLS Regression Results                              
Dep. Variable:     np.log(price_per_sqft)   R-squared:                       0.419
Model:                                OLS   Adj. R-squared:                  0.419
Method:                     Least Squares   F-statistic:                     5465.
Date:                    Thu, 06 Aug 2015   Prob (F-statistic):               0.00
Time:                            22:46:57   Log-Likelihood:                -30748.
No. Observations:                  105997   AIC:                         6.153e+04
Df Residuals:                      105982   BIC:                         6.167e+04
Df Model:                              14                                         
Covariance Type:                nonrobust                                         
                                   coef    std err          t      P>|t|      [95.0% Conf. Int.]
--------------------------------------------